In [2]:
%load_ext autoreload
%autoreload 2

In [109]:
import pandas as pd
from transformers import AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

In [110]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

### epoch 1, N_sup = 8

In [10]:
cd ..


/Users/bw/dev/knowledge-sep


In [169]:
df1 = pd.read_json('evals/checkpoint_epoch_1_Nsup8.jsonl', lines=True)

In [13]:
df1['is_match'].mean()

np.float64(0.195)

In [14]:
print(df1.iloc[7]['prompt'])

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>user
Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the beginning. How load does it take to download the file?<|im_end|>
<|im_start|>assistant



In [94]:
print(df1['generated_answer'].iloc[7])

<think>
The total time is 200/2 = <<200/2=100>>100 minutes.
The first 100 minutes is 100 minutes.
The second 20 minutes is 20 minutes.
So the total time is 100+20 = <<120>>120 minutes.
</think>

Answer: 120 minutes<|im_end|>
<|endoftext|>The total time to download the 200-GB file is 120 minutes.<|im_end|>
<|endoftext|>In a 200-GB file, Carla can download 200/2 = <<200/2=100>>100 minutes on the first 100 minutes of the file. 40% of the time is 20 minutes of the second 20 minutes of the 200-GB file. 100+20 = <<120>>120 minutes in total.<|im_end|>
<|endoftext|>In a 200-GB file, the total time to be downloaded is 120 minutes.<|im_end|>
<|endoftext|>Answer: 120<|im_end|>
<|endoftext|>Answer: 120<|im_end|>
<|endoftext|>


In [16]:
df1['generated_answer'].apply(lambda ga: ga.split(tokenizer.special_tokens_map['eos_token'])[0].strip()).iloc[7]

'<think>\nThe total time is 200/2 = <<200/2=100>>100 minutes.\nThe first 100 minutes is 100 minutes.\nThe second 20 minutes is 20 minutes.\nSo the total time is 100+20 = <<120>>120 minutes.\n</think>\n\nAnswer: 120 minutes'

In [17]:
def process_answer(ga):
    # Cut off parts after EOS token
    ga = ga.split(tokenizer.special_tokens_map['eos_token'])[0].strip()

    # Extract the final answer after the last "Answer:" marker
    if "Answer:" in ga:
        ga = ga.split("Answer:")[-1].strip()
        ga = ga.split(' ')[0].strip()
    return ga

df1['new_final_answer'] = df1['generated_answer'].apply(process_answer)

In [18]:
df1.iloc[7][['final_answer', 'new_final_answer']]

final_answer        120
new_final_answer    120
Name: 7, dtype: object

In [19]:
from data import get_generation_dataloader, get_dataloader

In [159]:
eval_loader = get_generation_dataloader(
    tokenizer, max_length=32768, batch_size=1,
    seed=42, train=False, num_samples=None
)

Loading dataset for generation evaluation...


In [164]:
def new_is_match(row):
    gt = row['ground_truth']
    new_ans = row['new_final_answer']
    try:
        new_is_match = (float(gt) == float(new_ans))
    except:
        new_is_match = (gt.lower() == new_ans.lower())
    return new_is_match

#### Does not include the <think> token in prompt!!

In [21]:
batch = None
for example in eval_loader:
    batch = example
    break

In [22]:
batch['prompts']

["<|im_start|>user\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|im_end|>\n<|im_start|>assistant\n",
 '<|im_start|>user\nA robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|im_end|>\n<|im_start|>assistant\n']

In [23]:
prompt_texts = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=False)

In [24]:
prompt_texts

["<|im_start|>user\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|im_end|>\n<|im_start|>assistant\n",
 '<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>user\nA robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|im_end|>\n<|im_start|>assist

In [66]:
example = {'question': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?'}

In [69]:
messages = [
    {'role': 'user', 'content': example['question']}
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

In [70]:
print(prompt)

<|im_start|>user
A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|im_end|>
<|im_start|>assistant



In [56]:
# trn_loader = get_dataloader(
#     tokenizer, 32768, 2,
#     seed=42, train=True, num_samples=None
# )

Loading dataset...


In [11]:
ground_truths = []
for i, example in enumerate(eval_loader):
    if i >= 200:
        break
    ground_truths.append({
        'id': i,
        'ground_truth': example['ground_truths'][0],
    })

In [12]:
gt_df = pd.DataFrame(ground_truths)

In [23]:
df1['ground_truth'] = [r['ground_truth'] for r in ground_truths]

In [29]:
df1['new_is_match'] = df1.apply(new_is_match, axis=1)

In [31]:
df1['new_is_match'].mean()

np.float64(0.195)

In [39]:
df1.head()

,id,prompt,generated_answer,final_answer,is_match,new_final_answer,ground_truth,new_is_match
0,1,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of eggs she eats in ...,16,False,16,18,False
1,2,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of bolts of fabric i...,<think>\nThe total number of bolts of fabric i...,False,<think>\nThe total number of bolts of fabric i...,3,False
2,3,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nThe total cost of the house is $80,00...","65,000",False,"65,000",70000,False
3,4,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of meters he runs in...,180,False,180,540,False
4,5,"<|im_start|>user\nEvery day, Wendi feeds each ...",<think>\nThe total number of cups of feed that...,30,False,30,20,False


In [45]:
row = df1.iloc[3]
print(row['prompt'])

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>

In [43]:
print(row['generated_answer'])

<think>
The total cost of the house is $80,000 + $50,000 = $<<130,000>>130,000.
The value of the house is 150% of the total cost, so 150/100 * $130,000 = $<<195,000>>195,000.
The profit is the total cost of the house and the value of the house, so $195,000 - $130,000 = $<<65,000>>65,000.
</think>

Answer: 65,000<|im_end|>
<|endoftext|>In a 150% value increase, the total cost of the house is $80,000 + $50,000 = $130,000.
The value of the house is 150% of the total cost, so 150/100 * $130,000 = $195,000.
The profit is the total cost of the house


### epoch 1, N_sup = 1

In [32]:
df2 = pd.read_json('evals/checkpoint_epoch_1_Nsup1.jsonl', lines=True)

In [33]:
df2['is_match'].mean()

np.float64(0.2)

In [34]:
df2['new_final_answer'] = df2['generated_answer'].apply(process_answer)

In [35]:
df2['ground_truth'] = [r['ground_truth'] for r in ground_truths]

In [36]:
df2['new_is_match'] = df2.apply(new_is_match, axis=1)

In [38]:
df2['new_is_match'].mean()

np.float64(0.215)

### Base model

In [174]:
df3 = pd.read_json('evals/checkpoint_epoch_1_only_base.jsonl', lines=True)

In [175]:
df3

,id,prompt,generated_answer,final_answer,is_match
0,1,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. Janet's ducks lay 16...","<think>\nOkay, let's see. Janet's ducks lay 16...",False
1,2,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let me try to figure out this p...","<think>\nOkay, let me try to figure out this p...",False
2,3,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. Josh bought a house ...","<think>\nOkay, let's see. Josh bought a house ...",False
3,4,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, so James runs 3 sprints a week,...","<think>\nOkay, so James runs 3 sprints a week,...",False
4,5,"<|im_start|>user\nEvery day, Wendi feeds each ...","<think>\nOkay, let me try to figure out how ma...","<think>\nOkay, let me try to figure out how ma...",False
...,...,...,...,...,...
395,396,<|im_start|>user\nErica made 20 Valentine's ca...,"<think>\nOkay, let's see. Erica made 20 Valent...","<think>\nOkay, let's see. Erica made 20 Valent...",False
396,397,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. Jenny has a pizza wi...","<think>\nOkay, let's see. Jenny has a pizza wi...",False
397,398,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, so Mark has $50 in his bank acc...","<think>\nOkay, so Mark has $50 in his bank acc...",False
398,399,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let me try to figure out how ma...","<think>\nOkay, let me try to figure out how ma...",False


In [176]:
import re
def process_answer(ga):
    # Cut off parts after EOS token
    before_eos = ga.split(tokenizer.special_tokens_map['eos_token'])[0].strip()
    split_by_end_think = before_eos.split('</think>')
    # part before last </think> token
    thinking = '</think>'.join(split_by_end_think[:-1])
    # part after last </think> token
    final_answer = split_by_end_think[-1].strip()

    # Extract answer from **Final Answer:** ... \boxed{ans}
    BOXED_REGEX_OPTIONAL_FINAL_ANSWER = r'(?:\*{0,2}Final Answer:?\*{0,2})?\s*.*\\boxed\{(.+?)\}'
    BOXED_REGEX_REQUIRE_FINAL_ANSWER = r'\*{0,2}Final Answer:?\*{0,2}\s*.*\\boxed\{(.+?)\}'
    boxed_match = re.search(BOXED_REGEX_OPTIONAL_FINAL_ANSWER, final_answer, re.IGNORECASE)
    if boxed_match:
        final_answer = boxed_match.group(1).strip()
    else:
        boxed_match = re.search(BOXED_REGEX_REQUIRE_FINAL_ANSWER, thinking, re.IGNORECASE)
        if boxed_match:
            final_answer = boxed_match.group(1).strip()
    return final_answer
df3['new_final_answer'] = df3['generated_answer'].apply(process_answer)

In [177]:
tokenizer.special_tokens_map['eos_token'] ## <== THIS CHANGES DEPENDING ON WETHER BASE MODEL OF NOT!!!

'<|im_end|>'

In [178]:
df3.sample(10)

,id,prompt,generated_answer,final_answer,is_match,new_final_answer
352,353,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. So, Andrea saved som...","<think>\nOkay, let's see. So, Andrea saved som...",False,"<think>\nOkay, let's see. So, Andrea saved som..."
282,283,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. Maddy is buying pizz...","<think>\nOkay, let's see. Maddy is buying pizz...",False,"<think>\nOkay, let's see. Maddy is buying pizz..."
351,352,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. April is donating pl...","<think>\nOkay, let's see. April is donating pl...",False,10
59,60,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. The problem says a r...","<think>\nOkay, let's see. The problem says a r...",False,187
189,190,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let me try to figure out this p...","<think>\nOkay, let me try to figure out this p...",False,34
326,327,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. So the problem is ab...","<think>\nOkay, let's see. So the problem is ab...",False,14
365,366,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. There are 6 girls in...","<think>\nOkay, let's see. There are 6 girls in...",False,18
311,312,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. The problem is about...",**\n$$\n\boxed{91}\n$$,False,91
386,387,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let me try to figure out Wilson...","<think>\nOkay, let me try to figure out Wilson...",False,80
375,376,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nOkay, let's see. Mrs. Sherman made a ...","<think>\nOkay, let's see. Mrs. Sherman made a ...",False,48


In [179]:
eval_loader = get_generation_dataloader(
    tokenizer, max_length=32768, batch_size=1,
    seed=42, train=False, num_samples=None
)

Loading dataset for generation evaluation...


In [180]:
ground_truths = []
for i, example in enumerate(eval_loader):
    if i >= 400:
        break
    ground_truths.append({
        'id': i+1,
        'ground_truth': example['ground_truths'][0],
    })

In [181]:
len(ground_truths)

400

In [182]:
df3['ground_truth'] = [r['ground_truth'] for r in ground_truths]

In [183]:
df3['new_is_match'] = df3.apply(new_is_match, axis=1)

In [184]:
df3['new_is_match'].mean()

np.float64(0.6675)